In [9]:
import pandas as pd
import numpy as np

In [10]:
#https://alexlenail.medium.com/understanding-and-implementing-the-hypergeometric-test-in-python-a7db688a7458
matching_df = pd.read_csv('..\\Logs\\matching_results_fin.csv',index_col=0)

In [11]:
matching_df

,NodesPerBlock,Computed_EdgeCount,Computed_NodeCount,Real_EdgeCount,Real_NodeCount,Matches,MatchCount,Selection Method,Configuration,StructureLearningMethod
0,10.0,34.0,30.0,2.0,4.0,"[('P61981', 'P63027')]",1.0,OnPLs,No Config,HillClimb
1,10.0,34.0,30.0,2.0,4.0,"[('P61981', 'P63027')]",1.0,OnPLs,No Config,HillClimb
2,10.0,34.0,30.0,2.0,4.0,"[('P61981', 'P63027')]",1.0,OnPLs,No Config,HillClimb
3,10.0,34.0,30.0,2.0,4.0,"[('P61981', 'P63027')]",1.0,OnPLs,No Config,HillClimb
4,10.0,34.0,30.0,2.0,4.0,"[('P61981', 'P63027')]",1.0,OnPLs,No Config,HillClimb
...,...,...,...,...,...,...,...,...,...,...
0,5.0,19.0,16.0,0.0,0.0,[],0.0,OnPLs,No Config,HillClimb(6tiers)
0,5.0,17.0,15.0,2.0,4.0,[],0.0,OnPLs,No Config,HillClimb
0,15.0,50.0,45.0,14.0,17.0,"[('P49418', 'Q9BY11'), ('P61981', 'Q9Y2J2')]",2.0,OnPLs,No Config,HillClimb
0,15.0,50.0,45.0,14.0,17.0,"[('P49418', 'Q9BY11'), ('P61981', 'Q9Y2J2')]",2.0,OnPLs,No Config,HillClimb


Aggregating results for Method tested

Method 1 : HillClimb+NoWhitelist <br>
Method 2 : PC+NoWhitelist <br>
Method 3 : HillClimb+Whitelist <br>
Method 4 : HillClimb+NoWhitelist(Random selection)

In [12]:
from scipy.stats import hypergeom
# Helper Methods

def eval_significance(row, alpha, is_config=False):
    V = row['Computed_NodeCount']

    if is_config:
        v = V / 3
        m = (v * (v - 1)) / 2

        M2 = (V * (V - 1)) / 2
        M = M2 - 3 * m

    else:
        M = (V * (V - 1)) / 2  # the population size, aka all possible edges in a DAG with V nodes

    n = row['Real_EdgeCount']  # number of successes in the population aka how many 'real' edges there are
    N = row['Computed_EdgeCount']  # the sample size aka how many of the possible edges were actually present
    x = row['MatchCount']  # number of successes in sample aka  aka how many of the present edges are 'real' edges

    pval = hypergeom.sf(x - 1, M, n, N)

    # if pval < alpha:
    #     print('Matches found are  statistically significant, with p-value=', str(pval), ' and alpha=', str(alpha))
    # else:
    #     print('Matches found are NOT statistically significant!!! with p-value=', str(pval), ' and alpha=', str(alpha))

    return pval < alpha, pval


def separate_tests(df, alpha, is_config=False):
    total_pval = 0

    for idx, row in df.iterrows():
        _, pval = eval_significance(row, alpha, is_config)
        if ~np.isnan(pval):
            total_pval += pval
    print(df.drop(['Matches'], axis=1).mean(numeric_only=True))
    print('P-value obtained: ', str((total_pval / df.shape[0])))

    return total_pval / df.shape[0]


def setup_row(df):
    mean_row = np.round(df.drop(['Matches'], axis=1).mean(numeric_only=True))
    print(mean_row)
    return mean_row




In [13]:
# Method 1
matching_method11= matching_df.loc[(matching_df['NodesPerBlock'] == 5) & (matching_df['Configuration'] == 'No Config') & (matching_df['StructureLearningMethod'] =='HillClimb') & (matching_df['Selection Method']=='OnPLs')]
matching_method12 = matching_df.loc[(matching_df['NodesPerBlock'] == 15) & (matching_df['Configuration'] == 'No Config') & (matching_df['StructureLearningMethod'] =='HillClimb') & (matching_df['Selection Method']=='OnPLs')]
matching_method13 = matching_df.loc[(matching_df['NodesPerBlock'] == 25) & (matching_df['Configuration'] == 'No Config') & (matching_df['StructureLearningMethod'] =='HillClimb') & (matching_df['Selection Method']=='OnPLs')]

In [14]:
# Method 2
matching_method21= matching_df.loc[(matching_df['NodesPerBlock'] == 5) & (matching_df['Configuration'] == 'No Config') & (matching_df['StructureLearningMethod'] =='PC') & (matching_df['Selection Method']=='OnPLs')]
matching_method22 = matching_df.loc[(matching_df['NodesPerBlock'] == 15) & (matching_df['Configuration'] == 'No Config') & (matching_df['StructureLearningMethod'] =='PC') & (matching_df['Selection Method']=='OnPLs')]
matching_method23 = matching_df.loc[(matching_df['NodesPerBlock'] == 25) & (matching_df['Configuration'] == 'No Config') & (matching_df['StructureLearningMethod'] =='PC') & (matching_df['Selection Method']=='OnPLs')]

In [15]:
# Method 3
matching_method31 = matching_df.loc[(matching_df['NodesPerBlock'] == 5) & (matching_df['Configuration'] == 'Config(wb)') & (matching_df['StructureLearningMethod'] =='HillClimb') & (matching_df['Selection Method']=='OnPLs')]
matching_method32 = matching_df.loc[(matching_df['NodesPerBlock'] == 15) & (matching_df['Configuration'] == 'Config(wb)') & (matching_df['StructureLearningMethod'] =='HillClimb') & (matching_df['Selection Method']=='OnPLs')]
matching_method33 = matching_df.loc[(matching_df['NodesPerBlock'] == 25) & (matching_df['Configuration'] == 'Config(wb)') & (matching_df['StructureLearningMethod'] =='HillClimb') & (matching_df['Selection Method']=='OnPLs')]

In [16]:
matching_method41= matching_df.loc[(matching_df['NodesPerBlock'] == 5) & (matching_df['Configuration'] == 'No Config') & (matching_df['StructureLearningMethod'] =='HillClimb') & (matching_df['Selection Method']=='Random')]
matching_method42= matching_df.loc[(matching_df['NodesPerBlock'] == 15) & (matching_df['Configuration'] == 'No Config') & (matching_df['StructureLearningMethod'] =='HillClimb') & (matching_df['Selection Method']=='Random')]
matching_method43= matching_df.loc[(matching_df['NodesPerBlock'] == 25) & (matching_df['Configuration'] == 'No Config') & (matching_df['StructureLearningMethod'] =='HillClimb') & (matching_df['Selection Method']=='Random')]





HyperGeometric tests : Determine whether the amount of matches determined is high enough to not be found by chance.


In [17]:
alpha=0.1

separate_tests(matching_method11,alpha)
separate_tests(matching_method12,alpha)
separate_tests(matching_method13,alpha)

NodesPerBlock          5.000000
Computed_EdgeCount    17.000000
Computed_NodeCount    15.000000
Real_EdgeCount         1.980583
Real_NodeCount         3.961165
MatchCount             0.000000
dtype: float64
P-value obtained:  1.0
NodesPerBlock         15.000000
Computed_EdgeCount    49.991304
Computed_NodeCount    45.000000
Real_EdgeCount        13.930435
Real_NodeCount        16.921739
MatchCount             2.008696
dtype: float64
P-value obtained:  0.15869597596887466
NodesPerBlock         25.000000
Computed_EdgeCount    83.445545
Computed_NodeCount    75.000000
Real_EdgeCount        32.000000
Real_NodeCount        28.000000
MatchCount             2.188119
dtype: float64
P-value obtained:  0.21847621339777532


0.21847621339777532

In [18]:
#separate_tests(matching_method21,alpha)
separate_tests(matching_method22,alpha)
separate_tests(matching_method23,alpha)

NodesPerBlock         15.0
Computed_EdgeCount    10.0
Computed_NodeCount    17.0
Real_EdgeCount         0.0
Real_NodeCount         0.0
MatchCount             0.0
dtype: float64
P-value obtained:  1.0
NodesPerBlock         25.0
Computed_EdgeCount     9.0
Computed_NodeCount    17.0
Real_EdgeCount         0.0
Real_NodeCount         0.0
MatchCount             0.0
dtype: float64
P-value obtained:  1.0


1.0

In [19]:
separate_tests(matching_method31,alpha,True)
separate_tests(matching_method32,alpha,True)
separate_tests(matching_method33,alpha,True)

NodesPerBlock          5.0
Computed_EdgeCount    10.0
Computed_NodeCount    15.0
Real_EdgeCount         2.0
Real_NodeCount         4.0
MatchCount             0.0
dtype: float64
P-value obtained:  1.0
NodesPerBlock         15.00
Computed_EdgeCount    29.00
Computed_NodeCount    45.00
Real_EdgeCount        13.00
Real_NodeCount        16.00
MatchCount             0.08
dtype: float64
P-value obtained:  0.9573126711099902
NodesPerBlock         25.00
Computed_EdgeCount    49.00
Computed_NodeCount    75.00
Real_EdgeCount        31.00
Real_NodeCount        27.00
MatchCount             0.11
dtype: float64
P-value obtained:  0.9519210652878288


0.9519210652878288

In [20]:
separate_tests(matching_method41,alpha)
separate_tests(matching_method42,alpha)
separate_tests(matching_method43,alpha)

NodesPerBlock          5.000000
Computed_EdgeCount    16.775510
Computed_NodeCount    15.000000
Real_EdgeCount         0.091837
Real_NodeCount         0.173469
MatchCount             0.000000
dtype: float64
P-value obtained:  1.0
NodesPerBlock         15.000000
Computed_EdgeCount    49.096154
Computed_NodeCount    45.000000
Real_EdgeCount         3.961538
Real_NodeCount         5.692308
MatchCount             0.057692
dtype: float64
P-value obtained:  0.9548590674665317
NodesPerBlock         25.000000
Computed_EdgeCount    78.778947
Computed_NodeCount    75.000000
Real_EdgeCount        12.810526
Real_NodeCount        15.357895
MatchCount             0.126316
dtype: float64
P-value obtained:  0.9220085760884366


0.9220085760884366

Two Sample t-test: Determining whether the OnPLS selection method finds more matches than the Baseline/Random approach


In [21]:
import scipy.stats as stats

matches11 = np.array(matching_method11['MatchCount'])
matches41 = np.array(matching_method41['MatchCount'])
stats.ttest_ind(a=matches11,b=matches41,equal_var=True)


Ttest_indResult(statistic=nan, pvalue=nan)

In [22]:
matches12 = np.array(matching_method12['MatchCount'])
matches42 = np.array(matching_method42['MatchCount'])
stats.ttest_ind(a=matches12,b=matches42,equal_var=True)


Ttest_indResult(statistic=59.72682266884518, pvalue=1.0975978629169663e-136)

In [23]:
matches13 = np.array(matching_method13['MatchCount'])
matches43 = np.array(matching_method43['MatchCount'])
stats.ttest_ind(a=matches13,b=matches43,equal_var=True)

Ttest_indResult(statistic=36.73961701047505, pvalue=2.5690615562568985e-89)